In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import string


In [11]:
!ls ../../data_full/raw/twitter_data/IEEE

full_dataset-release.csv  reduced_dataset-release.csv


In [12]:
fp = "../../data_full/raw/twitter_data/IEEE/full_dataset-release.csv"
df = pd.read_csv(fp)


/tmp/ipykernel_3136/508669384.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


In [13]:
df = pd.read_csv("../../data_full/raw/twitter_data/IEEE/full_dataset-release.csv")
df2 = pd.read_csv("../../data_full/raw/twitter_data/IEEE/reduced_dataset-release.csv")

/tmp/ipykernel_3136/3309602408.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data_full/raw/twitter_data/IEEE/full_dataset-release.csv")
/tmp/ipykernel_3136/3309602408.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../../data_full/raw/twitter_data/IEEE/reduced_dataset-release.csv")


In [14]:
df_temp = df.copy()
df_temp = df_temp.dropna()
df_temp = df_temp.drop_duplicates()

count = df_temp['STOCK'].value_counts()
perc = count/count.sum()
dstr = pd.DataFrame({"Count": count, "Percentage": perc}).reset_index()
dstr

,index,Count,Percentage
0,Ford,52198,0.141989
1,Facebook,40837,0.111085
2,Google,33697,0.091663
3,Next,32606,0.088695
4,Apple,27940,0.076002
...,...,...,...
96,Volkswagen,18,0.000049
97,Equinor,12,0.000033
98,CVS Health,11,0.000030
99,21CF,4,0.000011


In [19]:
dstr.columns = ['STOCK', 'Count', 'Percentage']

In [20]:
# Top 5 stocks (~50% share of tweets)
TOP_STOCKS = dstr['STOCK'].head().to_list()

# Middle 25% 
MID_STOCKS = dstr['STOCK'].iloc[5:].head().to_list()

# Bottom 25% (Below Starbucks)
BOT_STOCKS = dstr['STOCK'].iloc[10:].to_list()

In [21]:
# COL = ['Ford', 'Facebook', 'Google']
df_top = df[df['STOCK'].isin(TOP_STOCKS) | df['TWEET'].isin(TOP_STOCKS)]
df_top = df_top.reset_index(drop=True)
l1 = len(df_top)
l2 = len(df_top[df_top['Unnamed: 0'].isnull()])
df_top[df_top['Unnamed: 0'].isnull()]
print(f"Expected cleaned df size: {l1 - l2}")

df_mid = df[df['STOCK'].isin(MID_STOCKS) | df['TWEET'].isin(MID_STOCKS)]
df_mid = df_mid.reset_index(drop=True)
l1 = len(df_mid)
l2 = len(df_mid[df_mid['Unnamed: 0'].isnull()])
df_mid[df_mid['Unnamed: 0'].isnull()]
print(f"Expected cleaned df size: {l1 - l2}")

df_bot = df[df['STOCK'].isin(BOT_STOCKS) | df['TWEET'].isin(BOT_STOCKS)]
df_bot = df_bot.reset_index(drop=True)
l1 = len(df_bot)
l2 = len(df_bot[df_bot['Unnamed: 0'].isnull()])
df_bot[df_bot['Unnamed: 0'].isnull()]
print(f"Expected cleaned df size: {l1 - l2}")

Expected cleaned df size: 187342
Expected cleaned df size: 89848
Expected cleaned df size: 90622


In [22]:
# Function to fix misalignment
def fix_misalignment(df):
    # Copy the original DataFrame to avoid altering it directly
    df = df.copy()

    # Create a mask where the first column is NaN
    mask = df.iloc[:, 0].isna()

    # Iterate over rows where the mask is True
    for idx in df[mask].index.tolist():  # Use a list of indices to avoid dynamic changes
        # Get the previous row index
        previous_idx = idx - 1
        
        # Ensure the previous index is valid and within bounds
        if previous_idx in df.index:  
            for col in range(1, len(df.columns)):
                # Check if the current value is non-NaN
                if not pd.isna(df.iloc[idx, col]):
                    # Ensure the column is within bounds of the previous row
                    if col+1 < len(df.columns):
                        df.iloc[previous_idx, col+1] = df.iloc[idx, col]

    # Drop rows where the first column is NaN
    df = df[~mask]

    return df


In [24]:
# 22 MINUTE RUNTIME
import time
# BOT
start_time = time.time()

df_bot_cleaned = fix_misalignment(df_bot)

end_time = time.time()
runtime = end_time - start_time
print(f'RUNTIME: {runtime:.2f} seconds')

# MID
start_time = time.time()

df_mid_cleaned = fix_misalignment(df_mid)

end_time = time.time()
runtime = end_time - start_time
print(f'RUNTIME: {runtime:.2f} seconds')

# TOP
start_time = time.time()

df_top_cleaned = fix_misalignment(df_top)

end_time = time.time()
runtime = end_time - start_time
print(f'RUNTIME: {runtime:.2f} seconds')

RUNTIME: 65.77 seconds
RUNTIME: 69.16 seconds
RUNTIME: 131.18 seconds


In [25]:
df_top_cleaned = df_top_cleaned.reset_index(drop=True)
df_top_cleaned = df_top_cleaned.drop('Unnamed: 0', axis=1)
df_top_cleaned

,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY
0,RT @IndiaHistorypic: 1994::Young @sundarpichai...,Google,31/01/2017,820.19,0.004438,0.030286,0.030286,0.035772,2020180.0,21.549,14.953,-1.0,0.0000
1,"RT @Google: ""If you have the feeling that some...",Google,31/01/2017,820.19,0.004438,0.030286,0.030286,0.035772,2020180.0,21.549,14.953,-1.0,-0.5500
2,"RT @IBMWatson: Welcome @Apple, #ACLU, #AAAI, @...",Apple,31/01/2017,121.35,0.002307,0.004944,0.004944,-0.011372,49200993.0,9.114,8.417,1.0,1.0000
3,RT @ColMorrisDavis: My family owned a @Ford de...,Ford,31/01/2017,12.36,0.000809,0.010518,0.010518,0.020227,46974479.0,26.321,27.647,1.0,0.0000
4,RT @ThatBitchFromNJ: Google's sponsored ad for...,Google,31/01/2017,820.19,0.004438,0.030286,0.030286,0.035772,2020180.0,21.549,14.953,-1.0,-0.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187337,RT @time2trade2: @McAllisterDen Trump called f...,Ford,30/09/2018,9.25,0.000000,0.000000,-0.002162,0.064865,30987233.0,23.492,22.989,1.0,0.0000
187338,Thanks Google\r\r\r\r\r\r\n\r\r\r\r\r\r\nHey L...,Ford,30/09/2018,9.25,0.000000,0.000000,-0.002162,0.064865,30987233.0,23.492,22.989,1.0,0.0000
187339,RT @rmbyrne: Do you have any colleagues who th...,Google,30/09/2018,1207.08,0.000000,0.000000,0.000232,-0.028962,1780759.0,15.005,17.588,-1.0,-0.0625
187340,RT @AnnCoulter: Item No. 3 for FBI to investig...,Ford,30/09/2018,9.25,0.000000,0.000000,-0.002162,0.064865,30987233.0,23.492,22.989,-1.0,0.0000


In [26]:
df_mid_cleaned = df_mid_cleaned.reset_index(drop=True)
df_mid_cleaned = df_mid_cleaned.drop('Unnamed: 0', axis=1)
df_mid_cleaned

,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY
0,"RT @nikitakhara: Thank you, @Starbucks CEO for...",Amazon,31/01/2017,823.48,0.008379,0.014924,0.014924,-0.001263,3137196.0,13.447,16.992,-1.0,0.800000
1,RT @cultcommoncore: Dumping @Starbucks\r\r\r\r...,Starbucks,31/01/2017,55.22,0.012314,0.016298,0.016298,0.058312,14307985.0,23.916,17.298,-1.0,0.000000
2,@amazon has your back when it comes to food sh...,Netflix,31/01/2017,140.71,0.003624,0.012366,0.012366,-0.004264,4411631.0,27.398,24.135,1.0,1.000000
3,"RT @nia4_trump: So instead of hiring 10,000 un...",Amazon,31/01/2017,823.48,0.008379,0.014924,0.014924,-0.001263,3137196.0,13.447,16.992,1.0,0.000000
4,Hmmm interesting... \r\r\r\r\r\r\nonly @IBM ha...,Amazon,31/01/2017,823.48,0.008379,0.014924,0.014924,-0.001263,3137196.0,13.447,16.992,-1.0,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89843,"I'm at Starbucks in Al-Andalus, Al Asimah http...",Starbucks,30/09/2018,56.84,0.000000,0.000000,0.008797,0.010732,8975955.0,17.035,13.123,-1.0,0.000000
89844,RT @suzford: This bodes well! #JulieRapaport n...,Amazon,30/09/2018,2003.0,0.000000,0.000000,0.004983,-0.043929,4085135.0,20.709,22.946,1.0,0.000000
89845,RT @shopWSS: Celebrating 90 years of Mickey Mo...,Nike,30/09/2018,84.72,0.000000,0.000000,-0.002125,0.009797,7452735.0,20.470,20.016,1.0,0.458333
89846,RT @radiodisney: Tune in to the Radio Disney I...,Disney,30/09/2018,116.94,0.000000,0.000000,-0.007696,-0.055926,7366846.0,15.232,13.230,1.0,0.000000


In [27]:
df_bot_cleaned = df_bot_cleaned.reset_index(drop=True)
df_bot_cleaned = df_bot_cleaned.drop('Unnamed: 0', axis=1)
df_bot_cleaned

,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY
0,@FAME95FM1 Jamaicans make money with @Payoneer...,PayPal,31/01/2017,39.78,0.002011,0.012318,0.012318,0.054801,9100057.0,18.769,16.099,-1.0,0.000000
1,@CBSi Jamaicans make money with @Payoneer @Pay...,PayPal,31/01/2017,39.78,0.002011,0.012318,0.012318,0.054801,9100057.0,18.769,16.099,1.0,0.000000
2,@Hitz92fm Jamaicans make money with @Payoneer ...,eBay,31/01/2017,31.83,0.010682,0.021363,0.021363,-0.057807,9469076.0,33.029,22.932,1.0,-0.166667
3,@gawker Jamaicans make money with @Payoneer @P...,CBS,31/01/2017,64.49,-0.001396,0.003877,0.003877,-0.023414,3523867.0,16.338,16.302,1.0,0.000000
4,@Dal_Schnur @DKuzLA @wrightco0704 @psuba98 @Re...,HSBC,31/01/2017,676.4,0.005322,0.016558,0.016558,-0.005322,27693980.0,12.549,15.009,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90617,RT @SyriaArchivist: @ReutersWorld How about Re...,Walmart,30/09/2018,93.91,0.000000,0.000000,0.002343,0.021191,6306274.0,8.702,11.667,-1.0,0.250000
90618,24 Destinations Where You Can Stay\r\r\r\r\r\r...,eBay,30/09/2018,33.02,0.000000,0.000000,0.009691,0.030890,8467405.0,22.738,17.202,1.0,0.000000
90619,"RT @DrKanyuira: In Japan, youths in their mid...",Honda,30/09/2018,3439.0,0.000000,0.000000,-0.009305,0.015121,5554600.0,26.410,22.868,-1.0,-0.063889
90620,RT @Spicywienerprod: The #sexy Roxy Cox Follo...,Reuters,30/09/2018,50.3139,0.000000,0.000000,0.000876,-0.006785,7988967.0,6.838,12.771,1.0,0.000000


In [29]:
!ls ../../data_full/processed/

In [30]:
fp = "../../data_full/processed/"

df_top_cleaned.to_csv(f'{fp}top.csv', index=False)
df_mid_cleaned.to_csv(f'{fp}mid.csv', index=False)
df_bot_cleaned.to_csv(f'{fp}bottom.csv', index=False)

In [36]:
df_top_cleaned["LSTM_POLARITY"].value_counts()

 1.0    106744
-1.0     80572
Name: LSTM_POLARITY, dtype: int64

In [35]:
df_top_cleaned["TEXTBLOB_POLARITY"].min()

-1.0

In [39]:
df_top_cleaned[["TEXTBLOB_POLARITY", "LSTM_POLARITY"]].sample(25)

,TEXTBLOB_POLARITY,LSTM_POLARITY
134190,0.400000,1.0
183422,-0.070000,-1.0
116244,0.136364,1.0
144737,-0.400000,1.0
5850,0.000000,1.0
166720,-0.050000,-1.0
61512,0.000000,1.0
5201,0.000000,1.0
75267,-0.200000,-1.0
153968,0.000000,1.0


In [44]:
df.sample(10)

,Unnamed: 0,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY
1006601,606577.0,RT @aasian: Nike watching y’all burn your Nike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539315,Next,21/07/2018,6050.0,0.0,0.013223,0.012893,0.009256,4.642590e+05,1.676800e+01,1.900900e+01,1.000,0.142857,NaN,NaN
1011150,609493.0,If this is how they react over Nike supporting...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1147641,697140,@JeffBezos This all is nonsense. How can you a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620285,Next,31/07/2018,5936.0,-0.012803234501347708,-0.011456,-0.011456,-0.001685,9.700390e+05,1.352600e+01,1.758800e+01,1.000,0.000000,NaN,NaN
92173,54171,RT @DevanF91: A thread: reason why I chose to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
772941,Facebook,14/08/2018,181.11,-0.005852796642924202,-0.004693,-0.004693,0.014908,1.910200e+07,3.091400e+01,6.867300e+01,1.000,0.000000,NaN,NaN
229824,141953.0,RT @EmFarl: Microsoft Word is the most sensiti...,Microsoft,27/10/2018,106.960000,0.000000,0.012528,-4.338070e-02,1.589379e-02,5.552310e+07,51.171,36.208000,1.0000,0.300000
1345530,NaN,Ford,27/09/2018,9.23,0.004334,0.017335,0.039003,6.283857e-02,5.727219e+07,2.455800e+01,23.023,-1.000000,-0.1125,NaN
301761,185999,"Spider-Man Homecoming, Ghost In The Shell &amp...",Shell,31/03/2017,2096.000000,0.012882,0.016221,7.156489e-03,-6.202290e-03,6.741165e+06,13.674,13.676000,1.0000,0.170455


In [27]:
stock_names1 = set(df["TWEET"])
stock_names2 = set(df["STOCK"])

In [45]:
stock_names3 = set(df["Unnamed: 0"])

In [46]:
len(stock_names1), len(stock_names2), len(stock_names3)

(547916, 4242, 1005532)

In [36]:
for idx,(stock1, stock2) in enumerate(zip(stock_names1, stock_names2)):
    if idx%5_00==0:
        print("_____________________")
        print(idx ,stock1)
        print(idx ,stock2)
        print("_____________________")
    if idx==100_000:break

_____________________
0 Check out JOHNNY WAS Los Angeles Royal Rose Polo 1414t Women's Shirt Size M #JohnnyWas #PoloShirt https://t.co/VBllV9nNuF via @eBay
0 5800.0
_____________________
_____________________
500 @ArmyStrang American Anarchist is on Netflix and it's all about the author. It's pretty wild and pretty boring but worth a watch.
500 239.4
_____________________
_____________________
1000 @Ryanair I have a cabin bag which is 50x35x24 
This is 5% smaller over all than your size BUT the depth is 4cm bigger. But overall the bag
1000 72.89
_____________________
_____________________
1500 @omarsar0 Google scholar!
1500 79.91
_____________________
_____________________
2000 https://t.co/SCZWvCtgVG
5 Cool things you can do with Facebook messenger via Facebook Customer Service
#CustomerServiceforFacebook
2000 413.5
_____________________
_____________________
2500 I posted a new video to Facebook https://t.co/Guf1dvAcQx
2500 103.6
_____________________
_____________________
3000 #UNet

## Creating set of stock names

In [98]:
set_of_stock_names = set()
set_of_dates = set()

In [99]:
def parse_date(stock_name, formats=("%d-%m-%Y", "%d/%m/%Y")): # CHECK FOR OTHER FORMATS
    for fmt in formats:
        try:
            return datetime.strptime(stock_name, fmt)  
        except ValueError:
            continue
    return None 

In [100]:
def is_float(stock_name):
    try:
        return float(stock_name)  
    except ValueError:
        return None

In [101]:
weird_symbols = ["@", "#", "*"]
for idx, stock in enumerate(stock_names1):
    # print(stock)
    if isinstance(stock, str): 
        words = stock.split()
        
        if len(words) < 2 and not any(words[0].startswith(symbol) for symbol in weird_symbols):
            stock_name = " ".join(words)
            
            # check for float and continue if true
            if is_float(stock_name) is not None:continue

            # cehck date and add if true
            date = parse_date(stock_name)  
            if date:
                set_of_dates.add(date)  

            # if not float or date, or tweet, it might be a stock name
            else:
                stock_name = stock_name.translate(str.maketrans('', '', string.punctuation))
                set_of_stock_names.add(stock_name)  
                
    # if idx == 10: break  
    

In [103]:
# set_of_stock_names # list(set_of_stock_names)[:10] 

In [104]:
len(set_of_stock_names ) # original 547916

90

In [75]:
# set_of_dates

In [105]:
len(set_of_dates )

97

In [113]:
weird_symbols = ["@", "#", "*"]
for idx, stock in enumerate(stock_names2):
    # print(stock)
    if isinstance(stock, str): 
        words = stock.split()
        
        if len(words) < 2 and not any(words[0].startswith(symbol) for symbol in weird_symbols):
            stock_name = " ".join(words)
            
            # check for float and continue if true
            if is_float(stock_name) is not None:continue

            # cehck date and add if true
            date = parse_date(stock_name)  
            if date:
                set_of_dates.add(date)  

            # if not float or date, or tweet, it might be a stock name
            else:
                stock_name = stock_name.translate(str.maketrans('', '', string.punctuation))
                print(stock_name)
                set_of_stock_names.add(stock_name)  
                
    # if idx == 50: break  

Viacom
Carrefour
Netflix
Amazon
BMW
Colgate
BlackRock
Samsung
Nike
Chevron
Nissan
salesforcecom
Groupon
Hyundai
Nestle
Thales
TripAdvisor
Ford
Mastercard
HP
Citigroup
Pepsi
Pfizer
Honda
Starbucks
HSBC
Microsoft
SAP
Expedia
GSK
JPMorgan
Ryanair
Burberry
ASOS
Comcast
ATT
Vodafone
Heineken
Sony
Tesco
Visa
Oracle
FedEx
Equinor
UPS
Kroger
IBM
Intel
PayPal
Yahoo
Reuters
Disney
PG
Next
AstraZeneca
BP
Allianz
21CF
Costco
Aviva
Google
CocaCola
Danone
Audi
Exxon
bookingcom
McDonalds
BASF
Boeing
Volkswagen
TMobile
easyJet
Siemens
CBS
Kelloggs
Toyota
adidas
LOreal
eBay
Cisco
Santander
Apple
Facebook
Walmart
Adobe
Gillette
Shell
Verizon
HM
Bayer


In [114]:
len(set_of_stock_names ) ,len(set_of_dates )

(90, 121)

In [115]:
weird_symbols = ["@", "#", "*"]
for idx, stock in enumerate(stock_names3):
    # print(stock)
    if isinstance(stock, str): 
        words = stock.split()
        
        if len(words) < 2 and not any(words[0].startswith(symbol) for symbol in weird_symbols):
            stock_name = " ".join(words)
            
            # check for float and continue if true
            if is_float(stock_name) is not None:continue

            # cehck date and add if true
            date = parse_date(stock_name)  
            if date:
                set_of_dates.add(date)  

            # if not float or date, or tweet, it might be a stock name
            else:
                stock_name = stock_name.translate(str.maketrans('', '', string.punctuation))
                print(stock_name)
                set_of_stock_names.add(stock_name)  

bookingcom
easyJet
Expedia
JPMorgan
Reuters
HM
Ford
Next
Microsoft
Boeing
Walmart
ATT
Cisco
TripAdvisor
GSK
Nestle
Adobe
Bayer
Visa
CocaCola
salesforcecom
httpstcorhHEMcQqMJ
httpstcouWh9T5YlBR
PayPal
21CF
Netflix
Samsung
Hyundai
BMW
SAP
Pepsi
Kroger
Thales
BASF
Carrefour
Siemens
BP
BlackRock
Viacom
Google
Heineken
Allianz
Gillette
Amazon
tha…
httpstcojckDXVZ…
Kelloggs
PG
LOreal
eBay
Nissan
Disney
Starbucks
Comcast
Toyota
Oracle
Costco
adidas
HP
Volkswagen
Ryanair
IBM
Danone
Tesco
Yahoo
Exxon
Intel
Shell
Vodafone
Honda
HSBC
Chevron
Groupon
Santander
ASOS
Sony
Aviva
CBS
Burberry
FedEx
Nike
Verizon
Audi
Equinor
Apple
Facebook
Colgate
htt…
Pfizer
show
TMobile
Mastercard
UPS
ActionThriller
AstraZeneca
McDonalds
Citigroup


In [116]:
len(set_of_stock_names ) ,len(set_of_dates )

(97, 121)

In [119]:
sorted(set_of_dates)

[datetime.datetime(2017, 1, 31, 0, 0),
 datetime.datetime(2017, 2, 1, 0, 0),
 datetime.datetime(2017, 2, 28, 0, 0),
 datetime.datetime(2017, 3, 1, 0, 0),
 datetime.datetime(2017, 3, 31, 0, 0),
 datetime.datetime(2017, 4, 1, 0, 0),
 datetime.datetime(2017, 4, 28, 0, 0),
 datetime.datetime(2017, 4, 29, 0, 0),
 datetime.datetime(2017, 5, 31, 0, 0),
 datetime.datetime(2017, 6, 1, 0, 0),
 datetime.datetime(2017, 6, 30, 0, 0),
 datetime.datetime(2017, 7, 1, 0, 0),
 datetime.datetime(2017, 7, 31, 0, 0),
 datetime.datetime(2017, 8, 1, 0, 0),
 datetime.datetime(2017, 8, 31, 0, 0),
 datetime.datetime(2017, 9, 1, 0, 0),
 datetime.datetime(2017, 9, 29, 0, 0),
 datetime.datetime(2017, 9, 30, 0, 0),
 datetime.datetime(2017, 10, 31, 0, 0),
 datetime.datetime(2017, 11, 1, 0, 0),
 datetime.datetime(2017, 11, 30, 0, 0),
 datetime.datetime(2017, 12, 1, 0, 0),
 datetime.datetime(2017, 12, 29, 0, 0),
 datetime.datetime(2017, 12, 30, 0, 0),
 datetime.datetime(2018, 7, 11, 0, 0),
 datetime.datetime(2018, 7, 

In [120]:
sorted(set_of_stock_names)

['21CF',
 'ASOS',
 'ATT',
 'ActionThriller',
 'Adobe',
 'Allianz',
 'Amazon',
 'Apple',
 'AstraZeneca',
 'Audi',
 'Aviva',
 'BASF',
 'BMW',
 'BP',
 'Bayer',
 'BlackRock',
 'Boeing',
 'Burberry',
 'CBS',
 'Carrefour',
 'Chevron',
 'Cisco',
 'Citigroup',
 'CocaCola',
 'Colgate',
 'Comcast',
 'Costco',
 'Danone',
 'Disney',
 'Equinor',
 'Expedia',
 'Exxon',
 'Facebook',
 'FedEx',
 'Ford',
 'GSK',
 'Gillette',
 'Google',
 'Groupon',
 'HM',
 'HP',
 'HSBC',
 'Heineken',
 'Honda',
 'Hyundai',
 'IBM',
 'Intel',
 'JPMorgan',
 'Kelloggs',
 'Kroger',
 'LOreal',
 'Mastercard',
 'McDonalds',
 'Microsoft',
 'Nestle',
 'Netflix',
 'Next',
 'Nike',
 'Nissan',
 'Oracle',
 'PG',
 'PayPal',
 'Pepsi',
 'Pfizer',
 'Reuters',
 'Ryanair',
 'SAP',
 'Samsung',
 'Santander',
 'Shell',
 'Siemens',
 'Sony',
 'Starbucks',
 'TMobile',
 'Tesco',
 'Thales',
 'Toyota',
 'TripAdvisor',
 'UPS',
 'Verizon',
 'Viacom',
 'Visa',
 'Vodafone',
 'Volkswagen',
 'Walmart',
 'Yahoo',
 'adidas',
 'bookingcom',
 'eBay',
 'easyJet'